<a href="https://colab.research.google.com/github/sanjib7777/PyTorch-Practice/blob/main/Next_Word_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from torch.utils.data import DataLoader, Dataset
from nltk.tokenize import word_tokenize
import nltk

In [3]:
document = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can we contact you?
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies
I am Sanjib Shah
"""

In [4]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
# tokenize
tokens = word_tokenize(document.lower())

In [6]:
vocab ={'<UNK>':0}
for token in Counter(tokens).keys():
  if token not in vocab:
    vocab[token]=len(vocab)
len(vocab)

237

In [7]:
# extract sentences from data
input_sent = document.split('\n')

len(input_sent)

61

In [8]:
for sentence in input_sent:
  print(word_tokenize(sentence.lower()))

['about', 'the', 'program']
['what', 'is', 'the', 'course', 'fee', 'for', 'data', 'science', 'mentorship', 'program', '(', 'dsmp', '2023', ')']
['the', 'course', 'follows', 'a', 'monthly', 'subscription', 'model', 'where', 'you', 'have', 'to', 'make', 'monthly', 'payments', 'of', 'rs', '799/month', '.']
['what', 'is', 'the', 'total', 'duration', 'of', 'the', 'course', '?']
['the', 'total', 'duration', 'of', 'the', 'course', 'is', '7', 'months', '.', 'so', 'the', 'total', 'course', 'fee', 'becomes', '799', '*', '7', '=', 'rs', '5600', '(', 'approx', '.', ')']
['what', 'is', 'the', 'syllabus', 'of', 'the', 'mentorship', 'program', '?']
['we', 'will', 'be', 'covering', 'the', 'following', 'modules', ':']
['python', 'fundamentals']
['python', 'libraries', 'for', 'data', 'science']
['data', 'analysis']
['sql', 'for', 'data', 'science']
['maths', 'for', 'machine', 'learning']
['ml', 'algorithms']
['practical', 'ml']
['mlops']
['case', 'studies']
['you', 'can', 'check', 'the', 'detailed', 'sy

In [9]:
def text_to_indices(sentence, vocab):
  numerical_sent = []
  for token in sentence:
    if token in vocab:
      numerical_sent.append(vocab[token])
    else:
      numerical_sent.append(vocab['<UNK>'])

  return numerical_sent

In [10]:
text_to_indices('i am sanjib shah', vocab)

[86, 0, 17, 0, 0, 83, 17, 0, 0, 86, 0, 0, 83, 0, 17, 0]

In [11]:
input_num_sent = []
for sentence in input_sent:
  input_num_sent.append(text_to_indices(word_tokenize(sentence.lower()), vocab))

In [12]:
input_num_sent[59]

[86, 94, 235, 236]

In [13]:
training_sequence = []
for sentence in input_num_sent:
  for i in range(1, len(sentence)):
    training_sequence.append(sentence[:i+1])

In [14]:
training_sequence[:5]

[[1, 2], [1, 2, 3], [4, 5], [4, 5, 2], [4, 5, 2, 6]]

In [15]:
len_list = []
for sequence in training_sequence:
  len_list.append(len(sequence))

max(len_list)

62

In [16]:
padded_training_sequence = []
for sequence in training_sequence:
  padded_training_sequence.append([0]*(max(len_list) - len(sequence)) + sequence)


In [17]:
len(padded_training_sequence[52])

62

In [18]:
padded_training_sequence = torch.tensor(padded_training_sequence, dtype=torch.long)

In [19]:
padded_training_sequence.shape

torch.Size([689, 62])

In [20]:
X = padded_training_sequence[:,:-1]
y = padded_training_sequence[:,-1]

In [21]:
y.shape

torch.Size([689])

In [22]:
class CustomDataset(Dataset):
  def __init__(self, X, y):
    self.X =X
    self.y = y
  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self,index):
    return self.X[index], self.y[index]


In [23]:
dataset = CustomDataset(X,y)
len(dataset)

689

In [24]:
dataloader = DataLoader(dataset, batch_size = 32, shuffle=True)

In [25]:
for input, output in dataloader:
  print(output)

tensor([143, 160,   6,  30,  24,  65,   2,  36,   3, 221,  30, 193, 224, 116,
          8,  33,  27, 174,   2,  26, 152,  99,  80,  73, 199, 101,  25,  86,
          3, 165,  83,  30])
tensor([  6, 187,  33, 132,   8,  48, 132,  65,  54, 129,  58,  41,  28,  78,
        189,   5,  44, 203,   2, 101,   3,   2, 204, 236, 202, 182,  49,  85,
         68,  89,  45,   6])
tensor([ 17,  86,  23, 129,  46, 101,  22, 129,  27,  17,   2,   3, 111, 161,
        211,  30, 130, 123,  25,  36, 135, 202,  33, 129,  94,  27, 190,  78,
         94, 226,  81,  19])
tensor([ 86,  73,  45, 113,  19,   2,  65,  31, 194, 185, 148, 121, 127,  33,
         97, 102, 120,  46,   2, 104,  94,   5,  28,  30, 106, 130, 201, 106,
         24,  18, 103,  94])
tensor([ 82,  24, 193,  64, 143,  27,  14,   2,  60,  27, 205,  72,   5,  87,
         23,  78,  25, 134,  98, 160,  23, 112,  24,  21,  80, 124, 122, 193,
         28,  40,  30,  10])
tensor([ 30,  88,  72, 229,   2,  89,   2,  24,  98,  78,  81, 139, 193,  6

In [33]:
class LSTMModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, 100)
    self.lstm = nn.LSTM(100, 150, batch_first = True)
    self.fc = nn.Linear(150, vocab_size)

  def forward(self, x):
    embedded = self.embedding(x)
    intermediate_hidden_state, (final_hidden_state, final_cell_state) = self.lstm(embedded)
    output = self.fc(final_hidden_state.squeeze(0))
    return output



In [34]:
model = LSTMModel(len(vocab))

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [36]:
model.to(device)

LSTMModel(
  (embedding): Embedding(237, 100)
  (lstm): LSTM(100, 150, batch_first=True)
  (fc): Linear(in_features=150, out_features=237, bias=True)
)

In [37]:
epochs = 50
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate )

In [41]:

for epoch in range(epochs):
  total_loss = 0
  for batch_x, batch_y in dataloader:
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    optimizer.zero_grad()
    output = model(batch_x)
    loss = criterion(output, batch_y)
    loss.backward()
    optimizer.step()
    total_loss = total_loss + loss.item()

  print(f'Epoch: {epoch+1}, Loss: {total_loss:.4f}')


Epoch: 1, Loss: 60.6659
Epoch: 2, Loss: 53.3105
Epoch: 3, Loss: 47.3283
Epoch: 4, Loss: 40.9718
Epoch: 5, Loss: 36.1032
Epoch: 6, Loss: 31.6642
Epoch: 7, Loss: 27.5206
Epoch: 8, Loss: 24.1017
Epoch: 9, Loss: 21.2371
Epoch: 10, Loss: 18.7335
Epoch: 11, Loss: 16.3479
Epoch: 12, Loss: 14.3074
Epoch: 13, Loss: 12.6557
Epoch: 14, Loss: 11.4152
Epoch: 15, Loss: 10.2970
Epoch: 16, Loss: 9.2882
Epoch: 17, Loss: 8.3926
Epoch: 18, Loss: 7.7858
Epoch: 19, Loss: 7.2679
Epoch: 20, Loss: 6.6615
Epoch: 21, Loss: 6.0988
Epoch: 22, Loss: 5.7800
Epoch: 23, Loss: 5.3537
Epoch: 24, Loss: 5.0478
Epoch: 25, Loss: 4.8183
Epoch: 26, Loss: 4.5610
Epoch: 27, Loss: 4.3793
Epoch: 28, Loss: 4.3193
Epoch: 29, Loss: 3.9960
Epoch: 30, Loss: 3.8894
Epoch: 31, Loss: 3.7575
Epoch: 32, Loss: 3.7119
Epoch: 33, Loss: 3.5373
Epoch: 34, Loss: 3.4000
Epoch: 35, Loss: 3.2974
Epoch: 36, Loss: 3.2185
Epoch: 37, Loss: 3.1886
Epoch: 38, Loss: 3.0274
Epoch: 39, Loss: 3.0834
Epoch: 40, Loss: 2.9156
Epoch: 41, Loss: 2.9460
Epoch: 42,

In [44]:
def prediction(model, vocab, text):

  # tokenize
  tokenized_text = word_tokenize(text.lower())
  # text to numerical indices
  num_text = text_to_indices(tokenized_text, vocab)
  # padding
  padded_text =torch.tensor( [0]*(61-len(num_text))+num_text, dtype=torch.long).unsqueeze(0)
  # send to model
  output = model(padded_text)
  # predicted index
  value,index = torch.max(output, dim=1)

  # merge with text
  return text + " " + list(vocab.keys())[index]

In [53]:
prediction(model, vocab, "I am Sanjib")

'I am Sanjib shah'